# Arbeidsløype for grafkonversjon med Grew

1. [Søk etter setningsmønster i trebanken](#1-søk-etter-setningsmønster-i-trebanken)
2. [Skriv en konverteringsregel med mønsteret](#2-skriv-en-konverteringsregel-med-mønsteret)
3. Hvis flere mønstre/regler kreves for å konvertere riktig, velg en strategi som gir én riktig graf per setning (vanligvis `Onf`)
4. Visualiser en setningsgraf

In [ ]:
# Fix PATH to include OPAM binaries where grewpy_backend is located
import os

os.environ["PATH"] = os.environ["HOME"] + "/.opam/5.2.0/bin:" + os.environ["PATH"]

# Check if a norwegian spacy model exists in the virtual environment
try:
    import nb_core_news_md
except ModuleNotFoundError:
    print("Download a Norwegian spacy model to visualise graphs with Displacy")
    !python -m spacy download nb_core_news_md

In [ ]:
import grewpy
from grewpy import Corpus, CorpusDraft, Request

grewpy.set_config("ud")  # ud or basic

In [ ]:
from pathlib import Path

# Load the NDT treebank
treebank_file = "../UD_output.conllu"
corpus = Corpus(treebank_file)
draft = CorpusDraft(corpus)
n_sentences = corpus.count(Request())  # Empty request will match all sentences

print("Antall setninger i trebanken: ", n_sentences)

## 1. Søk etter setningsmønster i trebanken

- Finn ut hvor mange og hvilke setninger som matcher et bestemt mønster
- Stram inn mønsteret etter behov for å bare matche ønskede setninger ut fra hvilken NDT-til-UD-regel man skal skrive

Se Grew-dokumentasjonen for info om [`Request`](https://grew.fr/doc/request/) og syntaks for mønstrene (`pattern`-feltet i Request-objektet).

In [ ]:
# Søk etter mønster
testpattern = """
pattern {
    V [ upos=VERB ];
    N [ upos=NOUN ];
    e: V -[ SUBJ ]-> N; 
}
"""
"""
pattern {
    HEAD -> N1;
    e: HEAD -[conj]-> N2; 
  
}
"""

# Antall matchende sentinger
n_matches = grew.corpus_count(testpattern, corpus)
print(n_matches, "sentences match the pattern in the corpus \n")

# Treff på noder og kanter
corpus_results = grew.corpus_search(testpattern, corpus)
print("5 first matches in the corpus: ", corpus_results[:5])

# setningsID fra treffene
match_ids = [match["sent_id"] for match in corpus_results]

# Se på grafen til et av treffene
(sent_graph := grew.corpus_get(match_ids[0], corpus))

sent_info = [sent_graph[token] for token in sorted(sent_graph, key=int)]
words = [token[0]["form"] for token in sent_info[1:]]

# Hent ut de matchede nodene og kantene fra setningen
nodes = grew.search(testpattern, sent_graph)

print(f"""
Sentence: {sent_id}
Text: {" ".join(words)}

Annotations: {sent_info}

Matching nodes in the selected graph:
{nodes}
""")

# Skriv grafen til en fil
grew.save(sent_graph, "graph_sample.json")

### Visuell mønstermatching

Regelutikling kan også gjøres i Arborator, hvor vi har et privat prosjekt: [NDT_conversion_to_UD](https://arboratorgrew.elizia.net/#/projects/NDT_conversion_to_UD)


Mønstersøk i eksisterende UD-versjon kan også gjøres i Grew Match: https://universal.grew.fr/?corpus=UD_Norwegian-Bokmaal@2.16

## 2. Skriv en konverteringsregel med mønsteret

Legge til enkle transformasjoner i regelen og kjøre mot én eksempelsetning.
    - Spisse/stramme inn regelen for å redusere antallet resulterende grafer.
    - Når en regel konverterer en setning til bare én graf, og grafen ser riktig ut (ifølge UDs retningslinjer), kjør regelen på andre setninger som matchet samme mønster og sjekk at disse også er riktige.


In [ ]:
rule_name = "nominal_subj"
# regel med sammensatt mønster og kommando
test_rule = """rule nominal_subj { 
pattern {
    V [ upos=VERB ];
    N [ upos=NOUN ];
    e: V -[ SUBJ ]-> N; 
}
commands {
    del_edge e;
    add_edge V -[ nsubj ]-> N;
} }"""

# Last inn regelen som et graph rewriting system
grs_rule = grew.grs(test_rule)

# Kjør regelen på den utvalgte setningen
grew.run(grs_rule, sent_graph, rule_name)

## Visualiser setningsgrafer

In [ ]:
# Hent ut en bestemt setningsgraf med setnings-ID
sent_ids = corpus.get_sent_ids()
sent_id = sent_ids[0]  # type: ignore
graph = corpus.get(sent_id)

print(f"Sentence ID: {sent_id}")
print(f"Text: {graph.to_sentence()}")

In [ ]:
from IPython.display import HTML, SVG, display
from spacy import displacy
from spacy_conll import init_parser
from spacy_conll.parser import ConllParser

from ndt2ud.visualize import visualize_graph_displacy, visualize_graph_dot

output_name = f"graph_{sent_id}"
try:
    # graph.to_svg() has a known issue with some grewpy installations, but may work in some environments
    svg_result = graph.to_svg()
    Path(f"{output_name}.svg").open("w", encoding="utf-8").write(svg_result)
except Exception as e:
    conllstr = graph.to_conll()

    if "nlp" not in globals():
        nlp = ConllParser(init_parser("nb_core_news_md", "spacy"))

    doc = nlp.parse_conll_text_as_spacy(conllstr)  # type:ignore
    visual_graph = HTML(
        displacy.render(
            doc,
            style="dep",
            jupyter=False,
            options={"compact": False, "color": "green", "distance": 100.0},
        )
    )
    # visual_graph = HTML(visualize_graph_displacy(graph, nlp, output_name=output_name))
else:
    print(f"Using alternative visualization method with dot and graphviz")
    result = visualize_graph_dot(graph, output_name)
    visual_graph = SVG(filename=result)

display(visual_graph)

## Lagre regler i filer 

Lagre reglene i GREW-format i en `grs`-fil, med valgt strategi. 


## Konverter trebanken fra NDT til UD